In [21]:
!pip install openai
from pprint import pprint
import json
import time
from openai import OpenAI

In [6]:

OPEN_AI_KEY="YOUR_API_KEY_HERE"

In [4]:
client = OpenAI(api_key = OPEN_AI_KEY)

### Step 1: Cargar los archivos

In [7]:
file = client.files.create(
    file=open(r"C:\Users\HP\Desktop\CATO CURSOS-1-2024\GER-TI CATO1-2024\Libros\Disenando-la-propuesta-de-valor-Alexander-Osterwalder-Yves-Pig.pdf","rb"),
    purpose = "assistants"
)

print(file.id)

file-ED8m2oHKgFq24xLX8WXPAKUu


In [8]:
file_list = client.files.list()
print(file_list)

SyncPage[FileObject](data=[FileObject(id='file-ED8m2oHKgFq24xLX8WXPAKUu', bytes=25899584, created_at=1708457730, filename='Disenando-la-propuesta-de-valor-Alexander-Osterwalder-Yves-Pig.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-2ZmvqG0fkd7ToPOJyz39i3Yg', bytes=62813329, created_at=1706477619, filename='testing-business-ideas.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-3rpolhfGYf1lgCNNHP8gT1ui', bytes=31374957, created_at=1706477609, filename='Generacíon-de-modelos-de-negocio.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-618oY757TWoUSlMjc3VIoBnj', bytes=25899584, created_at=1706477605, filename='Disenando-la-propuesta-de-valor-Alexander-Osterwalder-Yves-Pig.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False)


In [9]:
file_id = "file-ED8m2oHKgFq24xLX8WXPAKUu"

### Step 2: Crear el asistente

In [12]:
assistant = client.beta.assistants.create(
    instructions ="Eres un asistente experto en diseño de propuesta de valor para productos nuevos",
    model = "gpt-4-1106-preview",
    tools = [{"type":"retrieval"}],
    file_ids = [file_id]
)

### Step 3: Crear el Thread

In [13]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_CslLKoWl5HimeweuOTd06iun', created_at=1708458533, metadata={}, object='thread', tool_resources=[])


### Step 4: Añadir un mensaje del usuario

In [14]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role ="user",
    content = " Cual es la porpuesta de valor del yogurt LC1"

)

### Step 5: Ejecuta al asistente para obtener la respuesta

In [18]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Porfavor refierete al usuario como Juanito, si la pregunta no es clara, pide aclracion"
)
print(run.id)

run_kvMDg8jnEHgzr6lNBjQwwpwt


### Step 6: Ver el estaus de la ejecución

In [22]:
while run.status not in["complete","failed"]:
    run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
    )
    
    print(run.status)
    time.sleep(10)

completed
completed
completed
completed


KeyboardInterrupt: 

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)

print(run_steps)

### Step 7: Cuando la ejecucion de la consulta esta completada

In [24]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [25]:
for each in messages:
    pprint(each.role + ":" +each.content[0].text.value)

('assistant:El documento que proporcionaste parece ser una sucesión de '
 'imágenes, y no he logrado encontrar un texto específico relacionado con la '
 'propuesta de valor del yogurt LC1. Puede ser que las imágenes contengan '
 'diagramas, modelos de negocio o conceptos relacionados con la propuesta de '
 'valor en general, pero no un ejemplo específico aplicado al yogurt LC1.\n'
 '\n'
 'Si el documento que has subido es el correcto y está relacionado con la '
 'pregunta, por favor, dime si hay algún detalle adicional que debería buscar '
 'dentro del mismo. Si es un error y tienes otro documento que pueda contener '
 'la información que buscas, te invito a subirlo.')
'user: Cual es la porpuesta de valor del yogurt LC1'


### Step 8: Limpiamos 

In [28]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit = "20"
)

print(my_assistants.data)

[Assistant(id='asst_mFE7WUXXs7oiOK4Tt8Sw2zOb', created_at=1708458391, description=None, file_ids=['file-ED8m2oHKgFq24xLX8WXPAKUu'], instructions='Eres un asistente experto en diseño de propuesta de valor para productos nuevos', metadata={}, model='gpt-4-1106-preview', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')]), Assistant(id='asst_EtFqMwpeDozJDPzqHb8qhK97', created_at=1706477671, description=None, file_ids=['file-2ZmvqG0fkd7ToPOJyz39i3Yg', 'file-3rpolhfGYf1lgCNNHP8gT1ui', 'file-618oY757TWoUSlMjc3VIoBnj'], instructions='Alex, eres un especialista en análisis y generación de modelos de negocios, te basas exclusivamente en lo expresado por Alexander Osterwalder en sus diferentes obras y entrevistas. También eres especialista en programación python con la API de open IA. \nPara el análisis de modelos de negocios nunca tomas en cuenta ninguna otra fuente que no sea un libro o una entrevista de Alexander Osterwalder. Siempre darás prioridad en tus respuestas en lo

In [29]:
response = client.beta.assistants.delete("asst_mFE7WUXXs7oiOK4Tt8Sw2zOb")
print(response)

AssistantDeleted(id='asst_mFE7WUXXs7oiOK4Tt8Sw2zOb', deleted=True, object='assistant.deleted')
